# 0 Load data


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from function import *
import glob

%load_ext autoreload
%autoreload 2

'''


key parameters:
 from 2020-2050 by year: {cumulative npv, cumulative cost}

'''


# 加载光伏数据，提取以下列：
# 
df_npv_time_series = pd.read_csv('data/5.1_npv_time_series_2020usd.csv')
df_npv_time_series.head()


,pixel_id,lat,lon,area_m2,E_yr_pixel,RCP_Category,Model,analysis_year,capital_cost,capital_cost_source,...,electricity_price_source,installed_capacity_kw,grid_generation_kwh,annual_revenue_usd,annual_om_cost_usd,annual_net_cash_flow_usd,initial_investment_usd,cumulative_npv_usd,project_year,years_from_start
0,0,25.004168,-107.54583,778152.929849,2.123755e+08,RCP2.6,AIM/CGE 2.2,2020,2480.526000,interpolated_adjusted,...,interpolated_adjusted,132285.998074,2.123755e+08,2.696855e+07,6.981685e+06,1.998687e+07,3.281389e+08,-3.281389e+08,2020,0
1,0,25.004168,-107.54583,778152.929849,2.123755e+08,RCP2.6,AIM/CGE 2.2,2030,1898.176628,interpolated_adjusted,...,interpolated_adjusted,132285.998074,2.123755e+08,3.288988e+07,6.975068e+06,2.591481e+07,2.511022e+08,-5.099490e+07,2030,10
2,0,25.004168,-107.54583,778152.929849,2.123755e+08,RCP2.6,AIM/CGE 2.2,2040,1645.497296,interpolated_adjusted,...,interpolated_adjusted,132285.998074,2.123755e+08,3.412164e+07,6.993621e+06,2.712802e+07,2.176763e+08,1.203989e+08,2040,20
3,0,25.004168,-107.54583,778152.929849,2.123755e+08,RCP2.6,AIM/CGE 2.2,2050,1390.046860,interpolated_adjusted,...,interpolated_adjusted,132285.998074,2.123755e+08,3.007204e+07,7.005102e+06,2.306694e+07,1.838837e+08,1.707116e+08,2050,30
4,0,25.004168,-107.54583,778152.929849,2.123755e+08,RCP2.6,GCAM 5.3,2020,2657.555407,interpolated_adjusted,...,interpolated_adjusted,132285.998074,2.123755e+08,2.104875e+07,9.765659e+06,1.128309e+07,3.515574e+08,-3.515574e+08,2020,0
